<a href="https://colab.research.google.com/github/almutareb/rag-based-llm-app/blob/main/create_langchain_vectorstores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install needed packages
!pip install -qU transformers accelerate einops langchain xformers bitsandbytes faiss-gpu sentence_transformers typing-extensions==4.8.0 unstructured chromadb

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
from langchain.document_loaders.recursive_url_loader import RecursiveUrlLoader

from bs4 import BeautifulSoup as Soup

urls = ["https://langchain-doc.readthedocs.io/en/latest","https://python.langchain.com/docs/get-started"]
docs = []
for url in urls:
  loader = RecursiveUrlLoader(url=url, max_depth=10, extractor=lambda x: Soup(x, "html.parser").text)
  docs.extend(loader.load())
#documents = loader.load()

<ipython-input-33-66a2e94a9cf8>:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  loader = RecursiveUrlLoader(url=url, max_depth=10, extractor=lambda x: Soup(x, "html.parser").text)


In [32]:
len(docs)

287

In [27]:
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma
from langchain.document_loaders import ReadTheDocsLoader
from langchain.text_splitter import HTMLHeaderTextSplitter
from langchain.text_splitter import MarkdownTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.utils import filter_complex_metadata
import time
from langchain.embeddings import HuggingFaceEmbeddings

FAISS_INDEX_PATH = "/content/lc-faiss-multi-mpnet-1000-markdown"

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    chunk_size = 500,
    chunk_overlap  = 50,
    length_function = len,
)

markdown_splitter = MarkdownTextSplitter(chunk_size=1000, chunk_overlap=100)

# Stage one: read all the docs, split them into chunks.
st = time.time()
print('Loading documents ...')
#loader = ReadTheDocsLoader(documents_path)
#docs = loader.load()
chunks = text_splitter.create_documents([doc.page_content for doc in documents], metadatas=[doc.metadata for doc in documents])
et = time.time() - st
print(f'Time taken: {et} seconds.')

#Stage two: embed the docs.
# use all-mpnet-base-v2 sentence transformer to convert pieces of text in vectors to store them in the vector store
model_name = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs
    )
#print(f'Loading chunks into faiss vector store ...')
#st = time.time()
#db_faiss = FAISS.from_documents(chunks, embeddings)
#db_faiss.save_local(FAISS_INDEX_PATH)
#et = time.time() - st
#print(f'Time taken: {et} seconds.')
print(f'Loading chunks into chroma vector store ...')
st = time.time()
persist_directory='/content/lc-chroma-multi-mpnet-500'
db_chroma = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
et = time.time() - st
print(f'Time taken: {et} seconds.')

Loading documents ...
Time taken: 0.8861393928527832 seconds.
Loading chunks into chroma vector store ...
Time taken: 0.23302841186523438 seconds.


In [24]:
len(chunks)

9587

In [28]:
!cp -r /content/lc-chroma* /content/drive/MyDrive/RAG_DB